<a href="https://colab.research.google.com/github/Yikang1020/Neurally-informed-model/blob/main/Kelly_DDM_modelfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy.io



In [ ]:
dat = scipy.io.loadmat('/content/drive/MyDrive/Kelly2021/Kelly2020/modeling/dat.mat')


In [ ]:
dat_n = dat['dat'][0][0][0]
dat_pd = dat['dat'][0][0][1]
dat_q = dat['dat'][0][0][2]
dat_p = dat['dat'][0][0][3]

qps = [0.1,0.3,0.5,0.7,0.9]

In [ ]:
def simul_DDM1(pvar, pfix, Sel):

  """
  % parameter vector is passed in as two separate ones, listing the fixed and the free ones
  % There are no fixed parameters in the DDM model fitting. We only coded it this way for similarity to NI modelling

  % simulation function for DDM1 (full mode but no drift bias)
  % parameters:
  % b = bound height. +b/-b = correct/incorrect bounds
  % z = starting point bias
  % d = drift rate
  % Ter = nondecision time
  % sz = intertrial range of starting point (range of uniform dist)
  % st = intertrial range of nondecision time - uniform dist
  % eta = between-trial var in drift rate
  % cond order: condnames = {'Easy' 'LoCoh' 'Deadline'};
  """

  pm = np.zeros([1,17]) # 15 parameters plus N and the seed
  pm[np.where(Sel==1)] = pvar 
  pm[np.where(Sel==0)] = pfix
  
  b = np.zeros(3)
  z = np.zeros(3)
  d = np.zeros(3)
  Ter = np.zeros(3)
  
  b[0],b[1],b[2],z[0],z[1],z[2],d[0],d[1],d[2],Ter[0],Ter[1],Ter[2],sz,st,eta,N,seed = [pm[i] for i in range(17)]

  N = N*np.ones([3,3])

  np.random.seed(seed)

  b_c = b # bound per regime ('c' (for condition) is counter var always used for regime)
  d_c = d # drift rate per regime. Seems odd cos d and b already are per regime, but designed to be easily changeable for other options (e.g. only one d.r. per coherence)
  z_vc = np.vstack((z,[0,0,0],-z)) # the mean starting point for each of the 9 total conditions (Regime x cue validity)

  s = 0.1

  dt = 0.002

  t = np.arange(-0.2,1.6,dt)

  for c in range(N.shape[1]): # Regime
    # Initialise
    rt = np.zeros([np.sum(N[:,c]),1])
    ch = np.zeros([np.sum(N[:,c]),1])
    val = np.vstack((np.ones(N[1,c],1),2*np.ones(N[2,c],1),3*np.ones(N[3,c],1)))

    for v in range(N.shape[0]): # validity
      for n in range(N[v,c]): # each single trial
        index  = n # just to figure out which row in the overall simulated data matrix we will be entering this trial
        for i in range(v-1):
          index = index + N[i,c]
        
        # simulate trial - 'tt' stands for 'this trial'
        dtt = d_c[c] + np.random.normal()*eta #drift rate variability around the mean for this condition
        btt = b_c[c] # bound
        Tertt = Ter[c] + (np.random.uniform()-0.5)*st # non decisoin time
        ztt = z_vc[v,c] + (np.random.uniform()-0.5)*sz # starting point + cumulative sum

        # Make noisy evidence for this trial:
        noisyEv = np.hstack((np.zeros(1,len(np.where(t<Tertt))),dtt*dt+np.random.normal(1,len(np.where(t>=Tertt)))*s*np.sqrt(dt)))
        DV = ztt + np.cumsum(noisyEv)

        t1 = t[np.where(DV>btt)[0]]
        t2 = t[np.where(DV<btt)[0]]

        if len(t1)==0 and len(t2)==0:   
          rt[index,0]=np.nan
          ch[index,0]=2
        elif len(t1)!=0 and len(t2)==0:
          rt[index,0]=t1
          ch[index,0]=1
        elif len(t1)==0 and len(t2)!=0:
          rt[index,0]=t2
          ch[index,0]=0
        else:
          if t1 <= t2:
            ch[index,0] = 1
          else:
            ch[index,0] = 0
          rt[index,0] = np.min(t1,t2)
    RT[c] = rt
    CH[c] = ch
    V[c] = val
  # simulated data matrix to be returned:
  simdat = np.array([])
  for c in range(N.shape(1)):
    simdat = np.vstack((simdat,np.hstack((c*np.ones(np.sum(N[:,c]),1),V[c],CH[c],RT[c]))))



In [ ]:
def simul_DDM2(pvar, pfix, Sel):

  """
  % DDM2: only one drift rate parameter PER COHERENCE!

  % simulation function for DDM1 (full mode but no drift bias)
  % parameters:
  % b = bound height. +b/-b = correct/incorrect bounds
  % z = starting point bias
  % d = drift rate
  % Ter = nondecision time
  % sz = intertrial range of starting point (range of uniform dist)
  % st = intertrial range of nondecision time - uniform dist
  % eta = between-trial var in drift rate

  % cond order: condnames = {'Easy' 'Lo-coh' 'Deadline'};
  """

  pm = np.zeros([1,19]) # 17 parameters plus N and the seed
  pm[np.where(Sel==1)] = pvar 
  pm[np.where(Sel==0)] = pfix
  
  b = np.zeros(3)
  z = np.zeros(3)
  d = np.zeros(2)
  dc = np.zeros(3)
  Ter = np.zeros(3)
  
  b[0],b[1],b[2],z[0],z[1],z[2],d[0],d[1],dc[0],dc[1],dc[2],Ter[0],Ter[1],Ter[2],sz,st,eta,N,seed = [pm[i] for i in range(19)]

  N = N*np.ones([3,3])

  np.random.seed(seed)

  # Here's we'll list a 3x3 matrix of mean bound, drift rate, start pt for all regimes x validities
  b_vc = np.vstack((b,b,b)) # bound only varies across regime of course
  d_vc = np.vstack((np.hstack((d[0]+dc[0],d[1]+dc[1],d[0]+dc[2])),np.hstack((d[0],d[1],d[0])),np.hstack((d[0]-dc[0],d[1]-dc[1],d[0]-dc[2])))) # note the difference here - d(1) used for regime 3
  z_vc = np.vstack((z,[0,0,0],-z)) 

  s = 0.1 #within-trial noise - scaling parameter

  dt = 0.002

  t = np.arange(-0.2,1.6,dt) # Full time range to simulate. 

  for c in range(N.shape[1]): # Regime
    
    rt = np.zeros([np.sum(N[:,c]),1])
    ch = np.zeros([np.sum(N[:,c]),1])
    val = np.vstack((np.ones(N[1,c],1),2*np.ones(N[2,c],1),3*np.ones(N[3,c],1)))

    for v in range(N.shape[0]):
      for n in range(N[v,c]): 
        index  = n 
        for i in range(v-1):
          index = index + N[i,c]
        
        # simulate trial - 'tt' stands for 'this trial'
        dtt = d_vc[v,c] + np.random.normal()*eta 
        btt = b_vc[v,c] 
        Tertt = Ter[c] + (np.random.uniform()-0.5)*st 
        ztt = z_vc[v,c] + (np.random.uniform()-0.5)*sz 

      
        noisyEv = np.hstack((np.zeros(1,len(np.where(t<Tertt))),dtt*dt+np.random.normal(1,len(np.where(t>=Tertt)))*s*np.sqrt(dt)))
        DV = ztt + np.cumsum(noisyEv)

        t1 = t[np.where(DV>btt)[0]]
        t2 = t[np.where(DV<btt)[0]]

        if len(t1)==0 and len(t2)==0:   
          rt[index,0]=np.nan
          ch[index,0]=2
        elif len(t1)!=0 and len(t2)==0:
          rt[index,0]=t1
          ch[index,0]=1
        elif len(t1)==0 and len(t2)!=0:
          rt[index,0]=t2
          ch[index,0]=0
        else:
          if t1 <= t2:
            ch[index,0] = 1
          else:
            ch[index,0] = 0
          rt[index,0] = np.min(t1,t2)
    RT[c] = rt
    CH[c] = ch
    V[c] = val
  # simulated data matrix to be returned:
  simdat = np.array([])
  for c in range(N.shape(1)):
    simdat = np.vstack((simdat,np.hstack((c*np.ones(np.sum(N[:,c]),1),V[c],CH[c],RT[c]))))


In [ ]:
def simul_DDM3(pvar, pfix, Sel):

  """
  % DDM3: This one has only one Ter

  % simulation function for DDM1 (full mode but no drift bias)
  % parameters:
  % b = bound height. +b/-b = correct/incorrect bounds
  % z = starting point bias
  % d = drift rate
  % Ter = nondecision time
  % sz = intertrial range of starting point (range of uniform dist)
  % st = intertrial range of nondecision time - uniform dist
  % eta = between-trial var in drift rate

  % cond order: condnames = {'Easy' 'Lo-coh' 'Deadline'};

  """

  pm = np.zeros([1,18]) # 16 parameters plus N and the seed
  pm[np.where(Sel==1)] = pvar 
  pm[np.where(Sel==0)] = pfix
  
  b = np.zeros(3)
  z = np.zeros(3)
  d = np.zeros(3)
  dc = np.zeros(3)
  Ter = np.zeros(1)
  
  b[0],b[1],b[2],z[0],z[1],z[2],d[0],d[1],d[2],dc[0],dc[1],dc[2],Ter[0],sz,st,eta,N,seed = [pm[i] for i in range(18)]

  N = N*np.ones([3,3])

  np.random.seed(seed)

  # Here's we'll list a 3x3 matrix of mean bound, drift rate, start pt for all regimes x validities
  b_vc = np.vstack((b,b,b)) # bound only varies across regime of course
  d_vc = np.vstack((np.hstack((d[0]+dc[0],d[1]+dc[1],d[2]+dc[2])),np.hstack((d[0],d[1],d[0])),np.hstack((d[0]-dc[0],d[1]-dc[1],d[2]-dc[2])))) # note the difference here - d(1) used for regime 3
  z_vc = np.vstack((z,[0,0,0],-z)) 

  s = 0.1 #within-trial noise - scaling parameter

  dt = 0.002

  t = np.arange(-0.2,1.6,dt) # Full time range to simulate. 

  for c in range(N.shape[1]): # Regime
    
    rt = np.zeros([np.sum(N[:,c]),1])
    ch = np.zeros([np.sum(N[:,c]),1])
    val = np.vstack((np.ones(N[1,c],1),2*np.ones(N[2,c],1),3*np.ones(N[3,c],1)))

    for v in range(N.shape[0]):
      for n in range(N[v,c]): 
        index  = n 
        for i in range(v-1):
          index = index + N[i,c]
        
        # simulate trial 
        dtt = d_vc[v,c] + np.random.normal()*eta 
        btt = b_vc[v,c] 
        Tertt = Ter[c] + (np.random.uniform()-0.5)*st 
        ztt = z_vc[v,c] + (np.random.uniform()-0.5)*sz 

      
        noisyEv = np.hstack((np.zeros(1,len(np.where(t<Tertt))),dtt*dt+np.random.normal(1,len(np.where(t>=Tertt)))*s*np.sqrt(dt)))
        DV = ztt + np.cumsum(noisyEv)

        t1 = t[np.where(DV>btt)[0]]
        t2 = t[np.where(DV<btt)[0]]

        if len(t1)==0 and len(t2)==0:   
          rt[index,0]=np.nan
          ch[index,0]=2
        elif len(t1)!=0 and len(t2)==0:
          rt[index,0]=t1
          ch[index,0]=1
        elif len(t1)==0 and len(t2)!=0:
          rt[index,0]=t2
          ch[index,0]=0
        else:
          if t1 <= t2:
            ch[index,0] = 1
          else:
            ch[index,0] = 0
          rt[index,0] = np.min(t1,t2)
    RT[c] = rt
    CH[c] = ch
    V[c] = val
  # simulated data matrix to be returned:
  simdat = np.array([])
  for c in range(N.shape(1)):
    simdat = np.vstack((simdat,np.hstack((c*np.ones(np.sum(N[:,c]),1),V[c],CH[c],RT[c]))))


In [ ]:
def simul_DDM4(pvar, pfix, Sel):

  """
  % simulation function for DDM4 (full model including drift bias)
  % parameters:
  % b = bound height. +b/-b = correct/incorrect bounds
  % z = starting point bias
  % d = drift rate
  % dc = drift rate bias
  % Ter = nondecision time
  % sz = intertrial range of starting point (range of uniform dist)
  % st = intertrial range of nondecision time - uniform dist
  % eta = between-trial var in drift rate

  % cond order: condnames = {'Easy' 'Lo-coh' 'Deadline'};

  """

  pm = np.zeros([1,20]) # 18 parameters plus N and the seed
  pm[np.where(Sel==1)] = pvar 
  pm[np.where(Sel==0)] = pfix
  
  b = np.zeros(3)
  z = np.zeros(3)
  d = np.zeros(3)
  dc = np.zeros(3)
  Ter = np.zeros(3)
  
  b[0],b[1],b[2],z[0],z[1],z[2],d[0],d[1],d[2],dc[0],dc[1],dc[2],Ter[0],Ter[1],Ter[2],sz,st,eta,N,seed = [pm[i] for i in range(20)]

  N = N*np.ones([3,3])

  np.random.seed(seed)

  # Here's we'll list a 3x3 matrix of mean bound, drift rate, start pt for all regimes x validities
  b_vc = np.vstack((b,b,b)) # bound only varies across regime of course
  d_vc = np.vstack((np.hstack((d[0]+dc[0],d[1]+dc[1],d[0]+dc[2])),np.hstack((d[0],d[1],d[0])),np.hstack((d[0]-dc[0],d[1]-dc[1],d[0]-dc[2])))) # note the difference here - d(1) used for regime 3
  z_vc = np.vstack((z,[0,0,0],-z)) 

  s = 0.1 #within-trial noise - scaling parameter

  dt = 0.002

  t = np.arange(-0.2,1.6,dt) # Full time range to simulate. 

  for c in range(N.shape[1]): # Regime
    
    rt = np.zeros([np.sum(N[:,c]),1])
    ch = np.zeros([np.sum(N[:,c]),1])
    val = np.vstack((np.ones(N[1,c],1),2*np.ones(N[2,c],1),3*np.ones(N[3,c],1)))

    for v in range(N.shape[0]):
      for n in range(N[v,c]): 
        index  = n 
        for i in range(v-1):
          index = index + N[i,c]
        
        # simulate trial 
        dtt = d_vc[v,c] + np.random.normal()*eta 
        btt = b_vc[v,c] 
        Tertt = Ter[c] + (np.random.uniform()-0.5)*st 
        ztt = z_vc[v,c] + (np.random.uniform()-0.5)*sz 

      
        noisyEv = np.hstack((np.zeros(1,len(np.where(t<Tertt))),dtt*dt+np.random.normal(1,len(np.where(t>=Tertt)))*s*np.sqrt(dt)))
        DV = ztt + np.cumsum(noisyEv)

        t1 = t[np.where(DV>btt)[0]]
        t2 = t[np.where(DV<btt)[0]]

        if len(t1)==0 and len(t2)==0:   
          rt[index,0]=np.nan
          ch[index,0]=2
        elif len(t1)!=0 and len(t2)==0:
          rt[index,0]=t1
          ch[index,0]=1
        elif len(t1)==0 and len(t2)!=0:
          rt[index,0]=t2
          ch[index,0]=0
        else:
          if t1 <= t2:
            ch[index,0] = 1
          else:
            ch[index,0] = 0
          rt[index,0] = np.min(t1,t2)
    RT[c] = rt
    CH[c] = ch
    V[c] = val
  # simulated data matrix to be returned:
  simdat = np.array([])
  for c in range(N.shape(1)):
    simdat = np.vstack((simdat,np.hstack((c*np.ones(np.sum(N[:,c]),1),V[c],CH[c],RT[c]))))

In [ ]:
## DDM1: Full DDM with drift rate and nondecision time allowed to vary across regime, but no drift bias.
# Note the DDMs are numbered not in order of running but from simplest to
# most complex, DDM4 being the fullest one
model = 'simul_DDM1'

#options = optimset('fminsearch') # how?

par_range1 = np.array([[0.01,0.3],  # bound b easy
        [0.01,0.3],  # b LoCoh
        [0.01,0.3],  # b DL
        [0,0.2],    # z[0] starting point bias
        [0,0.2],    # z[1]
        [0,0.2],    # z[2]
        [0,0.5],    # d[0] drift rate
        [0,0.5],    # d[1]
        [0,0.5],    # d[2]
        [0,0.7],    # Ter[0] non-decision time - called tnd in paper
        [0,0.7],    # Ter[1]
        [0,0.7],    # Ter[2]
        [0,0.5],    # sz % starting point variability
        [0,0.5],    # st % non decision time variability
        [0,0.5]  ])   # eta  % drift rate variability - called sd in paper 

In [ ]:
## GRID search: diverse values of all parameters but for now just evaluate G2 with no regime or priors effect
# This first step is to copmrehensively find any parameter vectors that are contenders, to set as starting vectors later

np.random.seed(0)

startpm = np.array([])

Sel = np.hstack((np.ones([1,15]),np.ones([1,2]))) # vector to SELect which parameters are free - in practice moot for DDM

savestring = 'grid_noRegPrior1'

k = 0

for st in np.arange(0,0.4,0.1):
  for eta in np.arange(0,0.5,0.05):
    for sz in np.arange(0,0.4,0.05):
      for ter in np.arange(0,0.4,0.05):
        for d in np.arange(0.05,0.5,0.05):
          Ethisb = np.array([])
          for b in np.arange(0.02,0.3,0.02):
            k = k + 1
            pm = np.hstack((b,b,b,0,0,0,d,d,d,ter,ter,ter,sz,st,eta,1000,seed)) 

            # E = Gsq(pm(Sel==1),pm(Sel==0),Sel,dat,qps,model)

            

array([0, 0, 0])